## Project Background

![NYC](NYC.jpg)
### Introduction
Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like .csv, .tsv, and .xlsx.

Recall that CSV, TSV, and Excel files are three common formats for storing data. Three files containing data on 2019 Airbnb listings are available to you:

### data/airbnb_price.csv

listing_id: unique identifier of listing
price: nightly listing price in USD
nbhood_full: name of borough and neighborhood where listing is located

### data/airbnb_room_type.xlsx 
This is an Excel file containing data on Airbnb listing descriptions and room types.

listing_id: unique identifier of listing
description: listing description
room_type: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

### data/airbnb_last_review.tsv
This is a TSV file containing data on Airbnb host names and review dates.

listing_id: unique identifier of listing
host_name: name of listing host
last_review: date when the listing was last reviewed

#### Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

What is the average price, per night, of an Airbnb listing in NYC?
How does the average price of an Airbnb listing, per month, compare to the private rental market?
How many adverts are for private rooms?
How do Airbnb listing prices compare across the five NYC boroughs?

### Importing all the necessary libraries

In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m
import sqlalchemy as sql
import datetime as dt
from fuzzywuzzy import process as p

### converting the datsets into pandas dataframe

In [158]:
reviews= pd.read_csv('reviews.tsv', sep='\t')
price= pd.read_csv('prices.csv')
room_types= pd.ExcelFile('room_types.xlsx').parse(0)

### Work to do to clean the datasets:
 

#### Exploring the 'reviews' datasets and what to clean there
1. 'host_name' column has Null/missing entries, this needs to be corrected
2. the date format in the last_review column should be corrected
3. the datatype of the 'last_review' column should be in datetime format

In [159]:
print(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


#### Cleaning the 'reviews' dataset

In [160]:
### filling the missing host names with 'No name' value
reviews['host_name']= reviews.host_name.fillna('No name')

### changing the date format of the last_review column
reviews['last_review']= pd.to_datetime(reviews.last_review)

#### Exploring the 'price' datasets and what to clean there
1. Values in the 'price' column is listed string, it should be listed as float
2. price column has an extras 'dollar' written after the string valuse, this extra 'dollar' word should be removed
3. Have a look if the 'price' column's value has any outlier, but it will only be possible to checka fter converting the datatype and removing the 'dollar' word


In [161]:
print(price.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


#### Cleaning the 'price' dataset

In [162]:
###removing the 'dollar' string from the price column 
price['price']=price.price.str.strip('dollars')

In [163]:
### changing the values in the price column from string to float
price['price']=price.price.astype('float')
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   listing_id   25209 non-null  int64  
 1   price        25209 non-null  float64
 2   nbhood_full  25209 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 591.0+ KB


#### Exploring the 'room_types' datasets and what to clean there
1. 'description' column has some null/missing valuse, it should be corrected
2. 'room_type' column should be in 'category' type and it should have 3 categories. The formatting of the spelling or the word formatting in the values of the column should be in the same same format so that we have only three categories. 

In [164]:
print(room_types.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


In [165]:
### Filling the missing description cells with 'No description provided' value
room_types= room_types.fillna({'description':'No description provided'})

###  changing the room_tyoe into category and capitalizing each category to drop categories into 3 categories
room_types['room_type']=room_types['room_type'].astype('category')
room_types['room_type']=room_types['room_type'].str.capitalize()


#### Merging three table on 'listing_id'

In [166]:
df_merged= pd.merge(price,reviews,on='listing_id')
df= pd.merge(df_merged,room_types,on='listing_id')

#### Removing the free_listings from the merged dataset

In [167]:
### finding the free listed lsitings
free_listing= df['price']==0
df=df.loc[~free_listing]

#### Calculating average price
average_price= round(df.price.mean(),2)
print(average_price)

#### calculating average price per month
df['avg_price']=df[["price"]] * 365 / 12
average_price_per_month= round(df["avg_price"].mean(), 2)

#### calculating the difference
difference=  round((average_price_per_month - 3100),2)
print(difference)

### calculating average price per room type
avg_price_room_type=round(df.groupby(['room_type'])['price'].mean().reset_index(),2)
print(avg_price_room_type)


141.82
1213.61
         room_type   price
0  Entire home/apt  197.17
1     Private room   81.67
2      Shared room   53.65


In [168]:

#### Calculating average price
average_price= round(df.price.mean(),2)
print(average_price)

#### calculating average price per month
df['month']= df.last_review.dt.strftime('%m')
avg_price_month= round(df.groupby(['month'])['price'].mean().reset_index(),2)
print(avg_price_month)

### calculating average price per room type
avg_price_room_type=round(df.groupby(['room_type'])['price'].mean().reset_index(),2)
print(avg_price_room_type)


141.82
  month   price
0    01  140.56
1    02  145.61
2    03  136.24
3    04  143.21
4    05  139.60
5    06  141.67
6    07  144.49
         room_type   price
0  Entire home/apt  197.17
1     Private room   81.67
2      Shared room   53.65


#### Calculating room frequencies

In [176]:
#### Number of rooms per room type
room_frequencies= df.room_type.value_counts()
print(room_frequencies)

Entire home/apt    13266
Private room       11351
Shared room          585
Name: room_type, dtype: int64


#### Calculating listing prices by NYC

In [170]:
#### Converting all the values of nbhood_full column into 5 Boroughs of NYC
boroughs_nyc= pd.DataFrame({"boroughs": ["Brooklyn", "Manhattan", "Queens", "Staten Island", "The Bronx"]})

for state in boroughs_nyc['boroughs']:
    matches_score = p.extract(state, df['nbhood_full'], limit=df.shape[0])
    for match in matches_score:
        if match[1] >= 80:
            df.loc[df['nbhood_full'] == match[0], 'nbhood_full'] = state
            
#### Calculating the average price of each borough of NYC
avg_price_borough= df.groupby('nbhood_full')['price'].mean()
print(avg_price_borough)

nbhood_full
Brooklyn         121.974665
Manhattan        184.000678
Queens            92.813947
Staten Island     86.044944
The Bronx         79.241033
Name: price, dtype: float64


### calculating the price range by borough

In [171]:
#### renaming the name of the column nbhood_full into boroughs
df= df.rename(columns={'nbhood_full':'boroughs','month':'reviewed_month'})
df.head()

### Calculatimng the price range by boroughs
ranges= [0,69,175,350,np.inf]
labels=['budget','average','expensive','extravagant']
df['price_range_labels']=pd.cut(df['price'], bins= ranges, labels= labels)
price_range_borough= df.groupby('boroughs')['price_range_labels'].count()
print(price_range_borough)





boroughs
Brooklyn         10460
Manhattan        10322
Queens            3456
Staten Island      267
The Bronx          697
Name: price_range_labels, dtype: int64


### The nuber of listings in the bronx borough based on price range

In [172]:
df.loc[df.boroughs=='The Bronx']['price_range_labels'].value_counts()

budget         381
average        286
expensive       25
extravagant      5
Name: price_range_labels, dtype: int64

In [179]:
### The nuber of listings in each borough based on price range
listing_frequencies_borough= df.groupby(['boroughs','price_range_labels'])['price_range_labels'].count()
print(listing_frequencies_borough)

boroughs       price_range_labels
Brooklyn       budget                3201
               average               5534
               expensive             1466
               extravagant            259
Manhattan      budget                1150
               average               5289
               expensive             3073
               extravagant            810
Queens         budget                1632
               average               1505
               expensive              291
               extravagant             28
Staten Island  budget                 124
               average                123
               expensive               20
               extravagant              0
The Bronx      budget                 381
               average                286
               expensive               25
               extravagant              5
Name: price_range_labels, dtype: int64


In [174]:
# Create first_reviewed, the earliest review date
first_reviewed = df["last_review"].dt.date.min()

# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

#### solutions from the cleaned dataset

In [182]:
print('average price:', average_price)
print('average price per month: ', average_price_per_month)
print('difference is: ', difference)
print('Room frequencies: ',room_frequencies)
print('first reviewed on: ', first_reviewed)
print('Last reviewed on: ', first_reviewed)
print('Frequencies of Listing per borough: ',listing_frequencies_borough)

average price: 141.82
average price per month:  4313.61
difference is:  1213.61
Room frequencies:  Entire home/apt    13266
Private room       11351
Shared room          585
Name: room_type, dtype: int64
first reviewed on:  2019-01-01
Last reviewed on:  2019-01-01
Frequencies of Listing per borough:  boroughs       price_range_labels
Brooklyn       budget                3201
               average               5534
               expensive             1466
               extravagant            259
Manhattan      budget                1150
               average               5289
               expensive             3073
               extravagant            810
Queens         budget                1632
               average               1505
               expensive              291
               extravagant             28
Staten Island  budget                 124
               average                123
               expensive               20
               extravagant        

#### Converting the cleaned dataset into a .csv file

In [183]:
df.to_csv('dataset.csv')
